## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]


In [4]:
y_df['Attrition'].value_counts()

,count
Attrition,
No,1233
Yes,237


In [5]:
y_df['Department'].value_counts()

,count
Department,
Research & Development,961
Sales,446
Human Resources,63


In [6]:
# Create a list of at least 10 column names to use as X data
Selected_Columns = attrition_df[['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
                                 'YearsSinceLastPromotion', 'NumCompaniesWorked']]


# Create X_df using your selected columns
X = Selected_Columns


# Show the data types for X_df
X.dtypes


,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

overtime_encoder = LabelEncoder()
X['overtime_encoded'] = overtime_encoder.fit_transform(X['OverTime'])
X = X.drop(['OverTime'], axis=1)
X.head()

<ipython-input-7-4a53a37e6291>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['overtime_encoded'] = overtime_encoder.fit_transform(X['OverTime'])


,Education,Age,DistanceFromHome,JobSatisfaction,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked,overtime_encoded
0,2,41,1,4,0,1,6,0,8,1
1,1,49,8,2,1,3,10,1,1,0
2,2,37,2,3,0,3,0,0,6,1
3,4,33,3,3,0,3,8,3,1,1
4,1,27,2,2,1,3,2,2,9,0


In [8]:
# Split the data into training and testing sets - Moved this line to after encoding
from sklearn.model_selection import train_test_split
X_train, X_test, y_attrition_train, y_attrition_test, y_department_train, y_department_test = train_test_split(X, y_df.Attrition, y_df.Department)


In [9]:
X.dtypes

,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64
overtime_encoded,int64


In [10]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [11]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_train_encoded = department_encoder.fit_transform(y_department_train.values.reshape(-1, 1))

#Transform the testing data using the fitted encoder
department_test_encoded = department_encoder.transform(y_department_test.values.reshape(-1, 1))

# Create two new variables by applying the encoder to the testing data
department_columns = department_encoder.get_feature_names_out(['Department'])

y_test_dept_encoded = pd.DataFrame(department_test_encoded, columns=department_columns)
y_train_dept_encoded = pd.DataFrame(department_train_encoded, columns=department_columns)

y_train_dept_encoded.shape
y_train_dept_encoded.head()

,Department_Human Resources,Department_Research & Development,Department_Sales
0,0.0,0.0,1.0
1,0.0,1.0,0.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,0.0,1.0


In [12]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_train_encoded = attrition_encoder.fit_transform(y_attrition_train.values.reshape(-1, 1))

#Transform the testing data using the fitted encoder
attrition_test_encoded = attrition_encoder.transform(y_attrition_test.values.reshape(-1, 1))

# Create two new variables by applying the encoder to the testing data
attrition_columns = attrition_encoder.get_feature_names_out(['Attrition'])

y_test_attrition_encoded = pd.DataFrame(attrition_test_encoded, columns=attrition_columns)
y_train_attrition_encoded = pd.DataFrame(attrition_train_encoded, columns=attrition_columns)

y_train_attrition_encoded.shape
#y_test_attrition_encoded.head()

(1102, 2)

## Create, Compile, and Train the Model

In [13]:
# Find the number of columns in the X training data
X_train.shape

# Create the input layer
input_layer = layers.Input(shape=(X.shape[1],), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(10, activation='relu')(input_layer)
shared_layer2 = layers.Dense(8, activation='relu')(shared_layer1)

In [23]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_dense = layers.Dense(10, activation='relu')(shared_layer2)

# Create the output layer
department_output = layers.Dense(len(y_train_dept_encoded.columns),
                                 activation='sigmoid',
                                 name='department_output')(department_dense)


In [24]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_dense = layers.Dense(10, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(len(y_train_attrition_encoded.columns),
                                 activation='sigmoid',
                                 name='attrition_output')(attrition_dense)


In [25]:
# Create the model
model = Model(inputs=input_layer, outputs=[
    department_output,
    attrition_output
])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'categorical_crossentropy'},
              metrics={'department_output': 'accuracy',
                       'attrition_output': 'accuracy',})

# Summarize the model
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_features            │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 10)             │            110 │ input_features[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 8)              │             88 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 10)             │             90 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 10)             │             90 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             33 │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             22 │ dense_8[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 433 (1.69 KB)

 Trainable params: 433 (1.69 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
# Train the model
model.fit(
    X_train_scaled,
    {
        'department_output': y_train_dept_encoded,
        'attrition_output': y_train_attrition_encoded },
    epochs=100, # You can adjust the number of epochs based on your needs
    batch_size=20,  # You can adjust the batch size based on your available memory
    validation_split=0.2  # You can specify the validation split if you have a separate validation set
)


Epoch 1/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - attrition_output_accuracy: 0.5964 - department_output_accuracy: 0.5286 - loss: 1.6970 - val_attrition_output_accuracy: 0.7511 - val_department_output_accuracy: 0.6154 - val_loss: 1.4763
Epoch 2/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.7879 - department_output_accuracy: 0.6633 - loss: 1.3979 - val_attrition_output_accuracy: 0.8054 - val_department_output_accuracy: 0.6335 - val_loss: 1.3860
Epoch 3/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8117 - department_output_accuracy: 0.6660 - loss: 1.3071 - val_attrition_output_accuracy: 0.8054 - val_department_output_accuracy: 0.6335 - val_loss: 1.3631
Epoch 4/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8025 - department_output_accuracy: 0.6615 - loss: 1.2738 - val_attrition_output_accuracy: 0.8054 - val_department_output_accuracy: 0.6335 - val_loss: 1.3525
Epoch 5/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5

In [27]:
# Evaluate the model with the testing data
results = model.evaluate(np.array(X_test), {
        'department_output': y_test_dept_encoded,
        'attrition_output': y_test_attrition_encoded }
    )

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8364 - department_output_accuracy: 0.6304 - loss: 5.2122  


In [30]:
# Print the accuracy for both department and attrition
pred_categories = ['Department', 'Attrition']
for i, cat in enumerate(pred_categories):
    print(f"{cat} predicted accuracy: {results[i+1]}")

Department predicted accuracy: 0.85597825050354
Attrition predicted accuracy: 0.6413043737411499


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.